In [22]:
import requests
import random
import json

base_url = "http://localhost:8081"

resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"


In [23]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][0]
jsonUrl

resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200


In [24]:
resp.json()

{'TrainingCertificate': {'$id': '#/properties/TrainingCertificate',
  'type': 'object',
  'title': 'The TrainingCertificate Schema',
  'required': ['name', 'contact'],
  'properties': {'name': {'type': 'string'},
   'trainingTitle': {'type': 'string'},
   'contact': {'type': 'string'},
   'date': {'type': 'string', 'format': 'date'},
   'note': {'type': 'string'}}}}

In [25]:
entities = list(resp.json().keys())
print("Available entities ", entities)

Available entities  ['TrainingCertificate']


In [26]:
entity_name=entities[0]
print("Using entity %s"%entity_name)


Using entity TrainingCertificate


In [27]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/%s'%entity_name), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module"
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"][entity_name]["osid"]


In [28]:
resp.json()


{'id': 'sunbird-rc.registry.create',
 'ver': '1.0',
 'ets': 1644476968108,
 'params': {'resmsgid': '',
  'msgid': '7dbbed5a-4538-4cf6-a862-afd44e40cb14',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'TrainingCertificate': {'osid': '1-21028fbe-d04b-4934-bfb8-7b78ee62ee31'}}}

In [29]:
print(resp.json())
resp.status_code, userId


{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1644476968108, 'params': {'resmsgid': '', 'msgid': '7dbbed5a-4538-4cf6-a862-afd44e40cb14', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'TrainingCertificate': {'osid': '1-21028fbe-d04b-4934-bfb8-7b78ee62ee31'}}}


(200, '14142570319')

In [30]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", "https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill"], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module"}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "Ed25519Signature2018", "created": "2022-02-10T07:09:29Z", "verificationMethod": "did:india", "proofPurpose": "assertionMethod", "jws": "eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..XO40XZ_3736m7FTqyAfSCA6AhBTpskcTK6OJk5QoSYD4GlZZFI98q7dtGQ4hGP_japdCd0aKvVfzH7vyGBQ-CA"}}


In [50]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf"})


In [51]:
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)
    

In [52]:
%system open 'sample.pdf'


[]

In [53]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"text/html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("<html")
with open('sample.html', 'wb') as f:
    f.write(resp.content)
%system open 'sample.html'


[]

In [54]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"image/svg+xml"})
resp.status_code, resp.content

assert resp.content.decode().find("<svg") != -1
with open('sample.svg', 'wb') as f:
    f.write(resp.content)
%system open 'sample.svg'

[]